In [1]:
import os
import sys
import re
os.getcwd()

'c:\\Users\\ebaca\\Desktop\\University of Washington\\3rd Year\\Astr-480\\Lab 4 - Data Reduction & Photometry'

In [2]:
sys.path.append('..\\..\\interacting_galaxies')
from funcrefs import fnrefs as rfs
from funcrefs import convenience_functions as cf

import numpy as np
import astropy.units as u

import matplotlib.pyplot as plt
import astropy.visualization as vis

from astropy.io import fits
from astropy.wcs import WCS
import ccdproc as ccdp
from astropy.nddata import CCDData

Notes for Observation Shifts
- for ngc 2998 if the picture given looks weird it's probably the spectrum pic
- darks are the same length as images you want to reduce (but you can also add some together)
- create a table that lists file names, filter, exposure time, and kind of file
- manually check for correct settings by downloading and opening fits file (don't trust preview screen)
- keep original data as raw
- [ccd data reduction guide](https://ccdproc.readthedocs.io/en/latest/)
- no overscans on ARCSAT

## <b>Data Reduction</b>

#### <u>Bias & Overscan</u>

1. <b>Define the bias. Include both a description of what it looks like physically, why it occurs, and how we take it into account (how do we remove its effect on our data images?)</b>

    A bias is a 0-second blank photo that records noise coming from the lens.

2. <b>Why would we use more than one bias frame?</b>

    The more bias frames you have the better you can remove noisy data from the pure photometric data. 
    This is helpful for being more precise with your values.

3. <b>List the average value of each of your bias frames, as well as your final master bias.</b>

    See below.

4. <b>Define "overscan".</b>

    

5. <s><b>Skip 5 (ARCSAT has no overscan)</b></s>

In [3]:
m_bias = fits.open('../../interacting_galaxies/biases/master_bias.fits')[0].data
m_bias = np.nan_to_num(m_bias.astype(float))

files = []
print("Average Bias Values (for each file)")
for file in os.listdir('../../interacting_galaxies/biases'):
    with fits.open('../../interacting_galaxies/biases/' + file) as hdul:
        data = hdul[0].data
        print(f"{np.mean(data):.3f}")
        files.append(np.mean(data))
print(f"Standard Deviation: {np.std(files):.3f}")

print(f"\nMaster Bias avg: {np.mean(m_bias):.3f}")

Average Bias Values (for each file)
1292.332
1292.311
1292.364
1292.303
1292.327
1292.282
1292.422
1292.429
1292.397
1292.422
1292.411
1292.365
1292.364
Standard Deviation: 0.048

Master Bias avg: 1292.364


#### <u>Flat Fields</u>

6. <b>Which filters do you have flat fields for? List the flat field file names for each filter that you need. Do you have more flats than you need? Confirm that your flat fields have the same binning as your data.</b>

    Filters: g, H-$\alpha$, i, r. 
    
    I'm not sure if there's a minimum amount of flats to have but in my opinion it doesn't hurt to have more! There are 5 flats for each filter. 
    
    See below for file names and confirming binning.

7. <b>Why do we need flat fields for each filter we have used for observations?</b>

    Similar to biases, these help to remove noise in the raw photometric data but are helpful for specifically pinpointing noise more prevalent in the correlating filter. Biases are more general and pertain to the noise coming from the telescope itself, where flats are for noise coming from the atmosphere or other parts of the sky.

In [9]:
master_flats = {}
for filter in ["g", "ha", "i", "r"]:
    m_flat = fits.open(f'../../interacting_galaxies/flats/{filter} flats/master_{filter}filter.fits')[0].data
    m_flat = np.nan_to_num(m_flat.astype(float))
    print(m_flat)
    # m_flat = rfs.normalize(m_flat, min(m_flat), max(m_flat))
    master_flats[filter] = m_flat


for filter in ["g flats", "ha flats", "i flats", "r flats"]:
    nm = [file for file in os.listdir(os.path.relpath("../../interacting_galaxies/flats/" + filter)) if file.startswith('domeflat')]

    print(f"\n[{filter}] ({len(nm)} files)")
    print("\n".join(nm))

[[18574.58333333 18196.33333333 18211.91666667 ... 18013.33333333
  17930.55555556 18025.52777778]
 [18534.02777778 18160.02777778 18303.13888889 ... 17917.16666667
  17871.52777778 18006.77777778]
 [18484.69444444 18163.77777778 18158.30555556 ... 18056.22222222
  17985.         18057.91666667]
 ...
 [18101.44444444 17715.58333333 17587.77777778 ... 18005.58333333
  18069.36111111 18028.16666667]
 [18135.61111111 17659.19444444 17753.25       ... 18044.55555556
  18214.         17979.61111111]
 [18141.47222222 17692.36111111 17696.97222222 ... 18236.69444444
  18198.72222222 18165.55555556]]
[[18451.72222222 18342.63888889 18295.44444444 ... 18075.44444444
  17989.66666667 18310.38888889]
 [18717.63888889 18095.02777778 18296.33333333 ... 18102.41666667
  18058.16666667 18054.13888889]
 [18586.5        18359.75       18618.44444444 ... 18078.05555556
  17957.75       18102.13888889]
 ...
 [17890.94444444 17590.77777778 17557.86111111 ... 17926.83333333
  17946.5        17940.55555556]

#### <u>Darks</u>

If your science images here are short, the dark current is extremely low and so we ignore it here. If you are taking longer images, or on instruments with lots of dark current, you can’t neglect this term of the noise in reduction. Make sure to take into account darks

In [ ]:
m_dark = fits.open('../../interacting_galaxies/darks/master_dark.fits')[0].data
m_dark = np.nan_to_num(m_dark.astype(float))

## <b>Data Analysis (Photometry)</b>

You now have images that are as close as we can get to what was emitted from our source. You can either use one of the broadband images from your project data or download the image from Canvas to use for this portion of the lab. It will provide an example for the different approaches to photometry you can then use for your project. (The next portion looks for point sources - if you use an image that has a diffuse image, like the nebulas, it might do some weird things for some of the options like DAOStarFinder. If you’re having trouble navigating that - just grab the image on canvas.)

We are going to use the [“photutils”](https://photutils.readthedocs.io/en/stable/) python package to measure the signal.

#### <u>The Sky</u>

We are going to be subtracting the sky on the fly as we use aperture photometry to calculate the flux of the stars in our field. If your primary science object is diffuse or extended just choose a stars in your field that aren’t overexposed/saturated as your object.

8. What drawbacks could exist from this method of sky subtraction?

9. Why does sky subtraction matter?

Identify the sources to perform photometry on in your image. DAOStarFinder is one 
method provided within photutils. Plot the stars found on top of your image. Adjust your parameters to select a smaller set of stars (Between 10-100ish. Depending on your image). Replot. Remove any “bad” stars from the sources array by hand (hot spots, stars on the edge, stars on the overscan)

10.  How many objects did you identify with DAOStarFinder? Look at your image. Do you think that’s a reasonable number? What did you change to adjust the number of stars detected? How many stars did you end up with?

Photutils provides many different kinds of apertures. We will use two here, but other cases (such as galaxies) might require apertures that aren’t round. First, use the “circular aperture” Choose your method and execute your aperture photometry. Review the output. Then plot your apertures on top of your image.  

11. Are you happy with your apertures? Why or why not? Save the list of sources for comparison.

Now we’re going to do the same thing, but instead of using just a circular aperture, we’re going to use annuli. This means you can specify a region for the star, and a nearby region for the sky. This allows us to use the sky that is close to the individual objects to subtract.

Pick your object aperture and note it in your Q12 answers. 
Pick your sky annulus minimum and maximum note it in your Q12 answers.

Now, run as above, except using the “CircularAnnulus” command. Use these two measurements of flux to remove the background from the originally measured flux, and print those results.

12. Make a plot comparing the measurements of the two different methods

Use PSF fitting to calculate the flux of the same set of sources you found with 
DAOstarfinder. Make sure to sky subtract

13. Make a plot comparing the PSF fitting output to the aperture output. Are they the same? Why or why not? 

14. What do you need to do differently for your science image? (For example - are you studying one point source, many point sources, or a diffuse object? What does that change?) 

15. Did you prefer PSF fitting or aperture photometry? Think of one situation that you think would be best for each method.